In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
# from implicit.evaluation import train_test_split
# from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


Подсчитываю объем продаж по каждому товару и получаю его вес: 
$$weight=\frac{[item\,quantity\,sum]}{[total\,quantity\,sum]}$$

In [6]:
sales_sum = data_train.groupby('item_id')['quantity'].sum().reset_index()
sales_sum.sort_values('quantity', ascending=False, inplace=True)
sales_sum.head()

,item_id,quantity
55470,6534178,190227964
55430,6533889,15978434
55465,6534166,12439291
55576,6544236,2501949
43620,1404121,1562004


In [7]:
weights_sum = sales_sum.quantity.sum()
sales_sum['weight'] = sales_sum.quantity.apply(lambda x: x / weights_sum)
sales_sum.drop('quantity', axis=1, inplace=True)
sales_sum.head()

,item_id,weight
55470,6534178,0.829765
55430,6533889,0.069697
55465,6534166,0.054260
55576,6544236,0.010913
43620,1404121,0.006813


Добавляю бейзлайн Weighted random recommendation

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights.item_id)
    weights = np.array(items_weights.weight)
    recs = np.random.choice(items, size=n, replace=False, p=weights)
        
    return recs.tolist()

In [9]:
%%time

result['weighted_random_recommendations'] = result['user_id'].apply(
                                lambda x: weighted_random_recommendation(sales_sum, n=5))
result.head()

Wall time: 4.8 s


,user_id,actual,weighted_random_recommendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 6534166, 895070, 5668996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6544236, 6534166, 397896, 6533889]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6534178, 397896, 6533889, 6534166, 1404121]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[6534178, 9704750, 1040371, 6534166, 6544236]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6533889, 6534178, 6534166, 5668996, 1404121]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [10]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


Добавляю бейзлайн Random recommendation, посчитанный на топ-5000 товаров

In [11]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [12]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
%%time

result['random_recommendations_top_5000'] = result['user_id'].apply(
                                lambda x: random_recommendation(top_5000, n=5))
result.head()

Wall time: 687 ms


,user_id,actual,weighted_random_recommendations,random_recommendations_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 6534166, 895070, 5668996]","[9296804, 885307, 6773215, 1047619, 5565664]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6544236, 6534166, 397896, 6533889]","[12384657, 854920, 948626, 989275, 882830]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6534178, 397896, 6533889, 6534166, 1404121]","[1056212, 1097001, 824180, 1039566, 5568509]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[6534178, 9704750, 1040371, 6534166, 6544236]","[12456256, 9836106, 1110793, 894425, 825343]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6533889, 6534178, 6534166, 5668996, 1404121]","[1090701, 944599, 1054383, 996028, 839753]"


Добавляю фиктивный item_id=6666

In [14]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [15]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

In [16]:
user_item_matrix.head()

item_id,6666,202291,397896,420647,480014,545926,707683,731106,818980,819063,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,707,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,255,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,164,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,138,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
user_item_matrix[user_item_matrix > 0] = 1
user_item_matrix = user_item_matrix.astype(float)

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [18]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Призвожу подбор параметра $k$ для ItemItemRecommender, CosineRecommender, TFIDFRecommender и BM25Recommender, дающий наилучший результат по метрике precision_at_k. Для расчета метрики использую функцию из домашнего задания к прошлому уроку.

In [19]:
def precision_at_k(recommended_list, bought_list, k=5):
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [20]:
params_dict = {
    'k': [],
    'item-item': [],
    'cosine': [],
    'tf-idf': [],
    'bm25': []
}

In [21]:
k_list = list(range(1, 11))

In [22]:
def get_models_precision(df, user_item_matrix, sparse_user_item, id_to_itemid, userid_to_id, 
                         itemid_to_id, k, n_recs=5, n_threads=4):
    ii_model = ItemItemRecommender(K=k, num_threads=n_threads)
    ii_model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
    df['item-item'] = df['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    ii_model.recommend(userid=userid_to_id[x], user_items=sparse_user_item, 
                    N=5, filter_already_liked_items=False, filter_items=[itemid_to_id[6666]], 
                    recalculate_user=True)])
    
    cos_model = CosineRecommender(K=k, num_threads=n_threads)
    cos_model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
    df['cosine'] = df['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    cos_model.recommend(userid=userid_to_id[x], user_items=sparse_user_item, 
                    N=5, filter_already_liked_items=False, filter_items=[itemid_to_id[6666]], 
                    recalculate_user=True)])
    
    tfidf_model = TFIDFRecommender(K=k, num_threads=n_threads)
    tfidf_model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
    df['tf-idf'] = df['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    tfidf_model.recommend(userid=userid_to_id[x], user_items=sparse_user_item, 
                    N=5, filter_already_liked_items=False, filter_items=[itemid_to_id[6666]], 
                    recalculate_user=True)])
    
    bm25_model = BM25Recommender(K=k, num_threads=n_threads)
    bm25_model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
    df['bm25'] = df['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    bm25_model.recommend(userid=userid_to_id[x], user_items=sparse_user_item, 
                    N=5, filter_already_liked_items=False, filter_items=[itemid_to_id[6666]], 
                    recalculate_user=True)])
    
    precision_dict = {}
    for name_col in df.columns[2:]:
        precision_dict[name_col] = round(df.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(), 4)
        
    return precision_dict

In [23]:
%%time

for k in k_list:
    res_df = result[['user_id', 'actual']]
    precision = get_models_precision(res_df, user_item_matrix, sparse_user_item, id_to_itemid, 
                        userid_to_id, itemid_to_id, k)
    
    params_dict['k'].append(k)
    for key in list(params_dict.keys())[1:]:
        params_dict[key].append(precision[key])

Wall time: 26.4 s


In [24]:
for key in list(params_dict.keys())[1:]:
    print(f'{key}. Best k={params_dict[key].index(max(params_dict[key])) + 1}, best precision: {max(params_dict[key])}')

item-item. Best k=1, best precision: 0.2199
cosine. Best k=2, best precision: 0.1763
tf-idf. Best k=2, best precision: 0.1795
bm25. Best k=4, best precision: 0.1337


Обучаю рекомендательные модели с получившимися значениями параметра $k$ 

ItemItemRecommender

In [25]:
%%time

model_ii = ItemItemRecommender(K=1, num_threads=4) 
model_ii.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)


Wall time: 527 ms


In [26]:
%%time

result['item_item'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_ii.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

Wall time: 51 ms


CosineRecommender

In [27]:
model_cos = CosineRecommender(K=2, num_threads=4)
model_cos.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

In [28]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_cos.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

Wall time: 75 ms


TFIDFRecommender

In [29]:
model_tfidf = TFIDFRecommender(K=2, num_threads=4)
model_tfidf.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

In [30]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_tfidf.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

Wall time: 70 ms


BM25Recommender

In [31]:
model_bm25 = BM25Recommender(K=4, num_threads=4)
model_bm25.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

In [32]:
%%time

result['bm25'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_bm25.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[6666]], 
                                    recalculate_user=True)])

Wall time: 82 ms


In [33]:
result.head()

,user_id,actual,weighted_random_recommendations,random_recommendations_top_5000,item_item,cosine,tfidf,bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 6534166, 895070, 5668996]","[9296804, 885307, 6773215, 1047619, 5565664]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 1127831, 961554, 979707, 1033142]","[1082185, 961554, 840361, 979707, 1033142]","[872137, 901062, 986947, 1082185, 914190]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6544236, 6534166, 397896, 6533889]","[12384657, 854920, 948626, 989275, 882830]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 1098066, 883404, 1044078, 826249]","[1082185, 1098066, 883404, 826249, 1044078]","[913202, 883404, 9526563, 833723, 10204822]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6534178, 397896, 6533889, 6534166, 1404121]","[1056212, 1097001, 824180, 1039566, 5568509]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 1127831, 878996, 981760, 844179]","[1082185, 1127831, 981760, 1044078, 878996]","[951412, 901062, 1024306, 1082185, 1119051]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[6534178, 9704750, 1040371, 6534166, 6544236]","[12456256, 9836106, 1110793, 894425, 825343]","[1082185, 995242, 1029743, 826249, 1127831]","[1082185, 1127831, 938700, 1013321, 961554]","[1082185, 1127831, 938700, 1013321, 1097544]","[7024990, 1030819, 5590613, 879635, 5587656]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6533889, 6534178, 6534166, 5668996, 1404121]","[1090701, 944599, 1054383, 996028, 839753]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 1098066, 1044078, 961554, 981760]","[1082185, 1098066, 981760, 1044078, 961554]","[870780, 1036432, 973673, 1053763, 903674]"


Проверяю качество получившихся моделей

In [34]:
for name_col in result.columns[1:]:
    print(f"{name_col}: {round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(), 4)}")

actual: 1.0
weighted_random_recommendations: 0.0482
random_recommendations_top_5000: 0.0054
item_item: 0.2199
cosine: 0.1763
tfidf: 0.1795
bm25: 0.1337


**Выводы:**
1. Бейзлайн "random recommendations on top-5000" показал результ лучше, чем просто "random recommendations", который считали на уроке.  
2. Бейзлайн weighted random recommendations показал результ существенно лучше, чем "random recommendations on top-5000" и "random recommendations", но хуже, чем "popular recommendation".
3. ItemItemRecommender показал наилучший результат, равный own_purchases, показанному на уроке.
4. CosineRecommender и TFIDFRecommender после побдора параметра $k$ улучшили качество, по равнению с результатом на уроке.
5. BM25Recommender на уроке не использовали. Увидел его в списке импортированных и решил попробовать. Он показал самый плохой результат из 4 моделей.